In [1]:
## 导入需要的库
import numpy as np
import pandas as pd
import math
import os
import random
from matplotlib.pyplot import *
import matplotlib.pyplot as plt

In [4]:
## 读取数据
df = pd.read_excel("中国科技巨头香港市场股票数据（2022 年）/香港预处理数据.xlsx")
df.head()

,股票代码,0,1,2,3,4,5,6,7,8,...,236,237,238,239,240,241,242,243,244,245
0,00020.HK,7.75,8.20,6.97,7.95,7.36,7.17,7.35,7.33,7.15,...,2.14,2.12,2.13,2.09,2.16,2.21,2.16,2.29,2.18,2.22
1,00700.HK,453.80,450.00,430.60,437.00,443.00,453.20,460.00,480.80,475.20,...,317.60,316.80,320.40,309.40,311.00,323.80,320.20,326.20,335.20,334.00
2,00992.HK,9.05,8.95,9.00,8.92,8.98,8.98,8.79,8.98,9.03,...,6.36,6.44,6.30,6.19,6.25,6.45,6.35,6.60,6.39,6.41
3,01024.HK,74.60,73.05,67.55,67.95,70.75,77.90,80.50,85.65,85.45,...,68.70,69.50,70.30,66.90,67.10,71.80,69.35,73.10,70.65,71.05
4,01810.HK,18.60,18.56,17.92,18.12,18.26,18.52,18.16,18.82,18.68,...,10.98,10.98,10.78,10.56,10.56,11.02,10.80,11.22,10.86,10.94


In [9]:
#根据日收盘价计算日收益率，再计算日对数收益率
df1 = np.zeros((len(df),len(df.T)-2))  #日收益率
df2 = np.zeros((len(df),len(df.T)-2))  #日对数收益率
for i in range(len(df)):
    for j in range(len(df.T)-2):
        df1[i][j] = (df.loc[i][j+1]-df.loc[i][j])/(df.loc[i][j])
        df2[i][j] = math.log(1+df1[i][j])
df2

array([[ 0.05644131, -0.16251893,  0.1315567 , ...,  0.0584436 ,
        -0.04922694,  0.01818232],
       [-0.00840899, -0.044068  ,  0.01475361, ...,  0.01856489,
         0.02721668, -0.00358638],
       [-0.01111123,  0.00557105, -0.00892863, ...,  0.03861484,
        -0.03233538,  0.003125  ],
       ...,
       [ 0.00901219, -0.04085075,  0.02063396, ...,  0.01065729,
        -0.0286758 ,  0.01533634],
       [ 0.02129398,  0.00971626,  0.00507615, ...,  0.02774991,
        -0.06299585,  0.02217386],
       [ 0.01638674, -0.02074405,  0.05521572, ...,  0.01377749,
        -0.0248209 ,  0.00814906]])

In [10]:
##根据算法对数据进行聚类
#归一化数据
n, m = len(df2), len(df2.T)
data1 = np.zeros((n,m))
for i in range(n):
    a = min(df2[i])
    b = max(df2[i])
    
    for j in range(m):
        data1[i][j]=(df2[i][j]-a)/(b-a)
data2 = pd.DataFrame(data1)
data2.to_csv(r"香港预处理后的数据归一化数据.csv")

In [11]:
def GYjuli(i,j):
    theta = 0.5
    m = len(df2.T)
    n = len(df2)
    a, b, a1, b1 = [], [], [], []
    X1 = np.zeros((1,m))
    X2 = np.zeros((1,m))
    A, B, A1, B1, D = 0, 0, 0, 0, 0
    # 取最大最小值
    for l in range(n):
        a.append(min(abs(data2.loc[i]-data2.loc[l])))
        b.append(max(abs(data2.loc[i]-data2.loc[l])))
        a1.append(min(abs(data2.loc[j]-data2.loc[l])))
        b1.append(max(abs(data2.loc[j]-data2.loc[l])))
    
    A, B = min(a), max(b)
    A1, B1 = min(a1), max(b1)
    c = abs(data2.loc[i]-data2.loc[j])
    
    for k in range(m):
        X1[0][k] = (A+theta*B)/(c[k]+theta*B) # 计算不同属性下的灰色关联系数
        X2[0][k] = (A1+theta*B1)/(c[k]+theta*B1)
    # print(X1)
    # print(X2)
    # D = np.sqrt(np.sum(np.square(X1 - X2))) #欧式距离
    D = np.sum(abs(X1 - X2)) #汉明距离
    return D

In [12]:
D = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        if i<j: 
            D[i][j] = GYjuli(i,j)
        if i>j:
            D[i][j] = D[j][i]
D1 = pd.DataFrame(D)
D1.to_csv(r"香港股票间的距离.csv")

In [13]:
def xiaoyong(x,singam,theta):
    lamda = np.zeros((len(x),6));
    lamda1 = np.ones((len(x),6));
    lamda2 = np.zeros((len(x),6));
    juhe = np.zeros((1,6));
    
    for i in range(0,len(x)): # 计算不同属性下的函数
        # 计算相对损失函数
        lamda[i,1] = singam * x[i];
        lamda[i,2] = x[i];
        lamda[i,3] = 1-x[i];
        lamda[i,4] = singam * (1-x[i]);
    
        # 计算基于后悔理论的效用函数
        lamda1[i,1] = 1 - (1 - np.exp(theta * lamda[i,1]))/theta
        lamda1[i,2] = 1 - (1 - np.exp(theta * lamda[i,2]))/theta
        lamda1[i,3] = 1 - (1 - np.exp(theta * lamda[i,3]))/theta
        lamda1[i,4] = 1 - (1 - np.exp(theta * lamda[i,4]))/theta
        
        # 计算相对效用函数
        lamda2[i,0] = 1 - lamda1[i,2] 
        lamda2[i,1] = lamda1[i,1] - lamda1[i,2] 
        lamda2[i,4] = lamda1[i,4] - lamda1[i,3] 
        lamda2[i,5] = 1 - lamda1[i,3]
        
    # 计算聚合相对效用函数(均值聚合)
    juhe = np.mean(lamda2,axis=0)
    
    return juhe

In [14]:
# 计算不同对象的聚合相对效用函数矩阵
singam = 0.15 
theta = 0.5
lamda = np.zeros((n,6));
for i in range(0,n):
    x = data2.iloc[i]
    lamda[i] = xiaoyong(x ,singam, theta)

#根据聚合相对效用函数计算不同对象的边界阈值
yuzhi = np.zeros((n,2))
for i in range(0,n):
    for j in range(0,len(lamda.T)):
        yuzhi[i][0] = (lamda[i][4])/(lamda[i][4]+lamda[i][0]-lamda[i][1])
        yuzhi[i][1] = (lamda[i][5]-lamda[i][4])/(lamda[i][1]+lamda[i][5]-lamda[i][4])
yuzhi = pd.DataFrame(yuzhi)
yuzhi.to_csv(r"香港股票的边界阈值.csv")

In [16]:
##根据归一化数据计算每个对象到正理想解和负理想解的距离
d1 = np.zeros((n,1)) 
d2 = np.zeros((n,1)) 
lishu = np.zeros((n,1))

for i in range(0,n):
    a = 0
    b = 0
    for j in range(0,m):
        a = a + (1 - data2.T[i][j])*(1 - data2.T[i][j])
        b = b + data2.T[i][j] * data2.T[i][j]
    d1[i] = np.sqrt(a) #每个对象到正理想解的距离
    d2[i] = np.sqrt(b) #每个对象到负理想解的距离
    lishu[i] = d1[i]/(d1[i]+d2[i])
lishuu = pd.DataFrame(lishu)
lishuu.to_csv(r"香港股票的隶属函数.csv")  

In [17]:
def qitayuzhi1(yz):
    #根据阈值设置下一个阈值的范围
    if yz[1] < 0.5:
        yuzhi2 = np.arange(yz[1],0.5,0.05) 
        V = []
        for k in range(0,len(yuzhi2)):
            yuzhi1 = 1-yuzhi2[k]
            x = 0 
            v1 = v2 = []
            v3 = 0
            
            #计算升高区域、降低区域、阴影区域的面积
            for i in range(0,n):
                if lishu[i] >= yuzhi1:
                    v1.append(1-lishu[i])
                elif lishu[i] <= yuzhi2[k]:
                    v2.append(lishu[i])
                else:
                    v3 = v3 + 1
            V1 = np.sum(v1)
            V2 = np.sum(v2)
            V3 = v3
    
            V.append(abs(V1+V2-V3))

    else:
        yuzhi2 = np.arange(yz[1],yz[0],0.05)
        x = 1
        V = []
        l = []
        
        for k in range(0,len(yuzhi2)):   
            yuzhi1 = np.arange(yuzhi2[k]+0.05,yz[0]+0.05,0.05)
            l.append(len(yuzhi1))
            for j in range(0,len(yuzhi1)):
                v1 = v2 = []
                v3 = 0
                
                #计算升高区域、降低区域、阴影区域的面积
                for i in range(0,n):
                    if lishu[i] >= yuzhi1[j]:
                        v1.append(1-lishu[i])
                    elif lishu[i] <= yuzhi2[k]:
                        v2.append(lishu[i])
                    else:
                        v3 = v3 + 1
                V1 = np.sum(v1)
                V2 = np.sum(v2)
                V3 = v3
    
                V.append(abs(V1+V2-V3))
        
    #根据最优化目标函数得到阈值
    if x == 0: 
        yz2 = yuzhi2[V.index(min(V))]
        yz1 = 1-yz2
    else:
        xx = V.index(min(V))
        for i in range(0,len(l)):
            if xx-l[i]>=0:
                xx = xx-l[i]
            else: 
                yz2 = yuzhi2[i]
                yuzhi1 = np.arange(yuzhi2[i]+0.05,yz[0]+0.05,0.05)
                yz1 = yuzhi1[xx]
    return yz1,yz2

In [18]:
df1 = pd.read_csv("香港股票的边界阈值.csv",index_col = 0)
df1.head()

,0,1
0,0.750234,0.067284
1,0.921827,0.219605
2,0.852471,0.119057
3,0.936596,0.259740
4,0.922566,0.219732


In [19]:
dis = pd.read_csv("香港股票间的距离.csv")
dis = dis.drop(['Unnamed: 0'],axis=1)
dis.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.000000,3.204688,5.965720,8.096169,3.523793,3.465203,3.537108,3.002339,2.941167,0.000000,0.371118
1,3.204688,0.000000,3.888643,2.147357,3.061028,2.901841,2.609443,0.050924,5.447549,1.392157,1.158666
2,5.965720,3.888643,0.000000,0.000000,9.653928,9.009536,9.928245,3.907907,7.173348,6.259580,6.377532
3,8.096169,2.147357,0.000000,0.000000,6.293652,6.817891,5.083439,2.574598,10.164632,4.313352,3.487343
4,3.523793,3.061028,9.653928,6.293652,0.000000,0.022855,0.020328,3.288066,0.035236,1.969331,1.835058


In [20]:
y = 0
for l in range(0,n):
    sc = [];uc = []; dc =[]; 
    # 根据边界阈值进行三支决策
    for i in range(n):
        if dis.loc[l][i]>=df1.loc[l][0]:
            dc.append(i)
        elif dis.loc[l][i]<df1.loc[l][0] and dis.loc[l][i]>df1.loc[l][1]:
            uc.append(i)
        else:
            sc.append(i) 
        
    # 根据其他阈值继续进行三支决策
    yz = qitayuzhi1(df1.loc[l])
    #print(yz)
    while y == 0:
        yuzhi = yz
        for i in range(n):
            if i in uc:
                if dis.loc[l][i] <= yuzhi[1]:
                    sc.append(i)
                    uc.remove(i)
                if dis.loc[l][i] >= yuzhi[0]: 
                    dc.append(i)
                    uc.remove(i)
        if yuzhi[0]- 0.5 > 0.1 or len(uc) >= 1/4*n:
            yz = qitayuzhi1(yuzhi)
            # print(yz)
        else:
            y = 1
    print(l,'............')        
    print(sc)
    print(uc)
    print(dc)

KeyboardInterrupt: 

In [22]:
y = 0
for l in range(0,n):
    sc = [];uc = []; dc =[]; 
    # 根据边界阈值进行三支决策
    for i in range(n):
        if dis.loc[l][i]>=df1.loc[l][0]:
            dc.append(i)
        elif dis.loc[l][i]<df1.loc[l][0] and dis.loc[l][i]>df1.loc[l][1]:
            uc.append(i)
        else:
            sc.append(i) 
    print(l,'............') 
    print(sc)
    print(uc)
    print(dc)

0 ............
[0, 9]
[10]
[1, 2, 3, 4, 5, 6, 7, 8]
1 ............
[1, 7]
[]
[0, 2, 3, 4, 5, 6, 8, 9, 10]
2 ............
[2, 3]
[]
[0, 1, 4, 5, 6, 7, 8, 9, 10]
3 ............
[2, 3]
[]
[0, 1, 4, 5, 6, 7, 8, 9, 10]
4 ............
[4, 5, 6, 8]
[]
[0, 1, 2, 3, 7, 9, 10]
5 ............
[4, 5, 6, 8]
[]
[0, 1, 2, 3, 7, 9, 10]
6 ............
[4, 5, 6, 8]
[]
[0, 1, 2, 3, 7, 9, 10]
7 ............
[1, 7]
[]
[0, 2, 3, 4, 5, 6, 8, 9, 10]
8 ............
[4, 5, 6, 8]
[]
[0, 1, 2, 3, 7, 9, 10]
9 ............
[0, 9, 10]
[]
[1, 2, 3, 4, 5, 6, 7, 8]
10 ............
[9, 10]
[0]
[1, 2, 3, 4, 5, 6, 7, 8]
